### 0. Advance Preparation

Please download the datasets from the site below and put them in the `data` directory in the directory in which this notebooks is.

__120 years of Olympic history: athletes and results__  
  URL:  
  ・https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results  
  DATA:  
  ・athlete_events.csv

---

__World cities database__  
  URL:  
    ・https://www.kaggle.com/juanmah/world-cities  
  DATA:  
    ・worldcities.csv

In [ ]:
%matplotlib inline

import folium
import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns

### 1. Datasets overview
Let's start with getting an overview of the datasets.

Read the athletes and results data file into DataFrame first.

In [ ]:
athlete_events_df = pd.read_csv('data/athlete_events.csv')

print(f"{len(athlete_events_df):,}件")

Check the contents.

If there's any column which you don't understand, please refer to the explanation on the page where you got the data.

In [ ]:
athlete_events_df.head()

Get the statistics of the data.

Do you find anything interesting?

In [ ]:
athlete_events_df.describe()

Make a histgram of the data.

Do you find anything interesting?

In [ ]:
athlete_events_df.drop(columns=['ID']).hist(bins=50, figsize=(20, 12)); # セミコロンでテキスト出力を抑える

To see multiple pairwise relationships in a dataset, you can use `pairplot()` function.

Do you find anything interesting?

In [ ]:
sns.pairplot(athlete_events_df.drop(columns=['ID']).sample(frac=0.5))

### 2. Datasets overview of locations with map

When position coordinates are available, plot them on a map.

Let put all the cities that have hold Olympic games on map according to the rules below.
- Put red circles for summer games, blue circles for winter games.
- Change the redius of the circle in proportion to number of participants.
- Put circles with slightly different colors when a city has held games multiple times.

To start with, group the data by cities and games.

In [ ]:
city_games_df = athlete_events_df.groupby(['City', 'Year', 'Games']).agg(number_of_athletes=('ID', 'count'))
city_games_df.reset_index(inplace=True)

city_games_df.head(10)

Add `season` column to know whether a game was held in summer or winter.

In [ ]:
city_games_df['season'] = city_games_df['Games'].str.split().str[1]

city_games_df.head(10)

Since some cities held games multiple times, we add `number_of_times` columns. 

In [ ]:
city_games_df['number_of_times'] = city_games_df.groupby(['City', 'season']).cumcount() + 1

city_games_df.head(10)

Read the world cities data file into DataFrame.

In [ ]:
worldcities_df = pd.read_csv('data/worldcities.csv')

len(worldcities_df)

Check the contents.

In [ ]:
worldcities_df.head(10)

We use `city_ascii` to get city names in English. 

Check whether there is any duplicate.

In [ ]:
worldcities_df.duplicated(subset='city_ascii').sum()

Now, we've got many duplicates.

Let's see what duplicates we've got.

In [ ]:
worldcities_df[worldcities_df.duplicated(subset='city_ascii')].sort_values('city_ascii')

When we find duplicates, we will choose the one with largest populations of them.

To do that, sort the data by city name and population and keep only the last row of each city name.

In [ ]:
worldcities_df.sort_values(['city_ascii', 'population'], inplace=True)

worldcities_df.drop_duplicates(subset='city_ascii', keep='last', inplace=True)

Check whether any duplicate is left.

In [ ]:
worldcities_df.duplicated(subset='city_ascii').sum()

Keep the only needed columms.

In [ ]:
worldcities_df = worldcities_df[['city_ascii', 'lat', 'lng']]

Join the two Dataframes on city name columns.

In [ ]:
city_game_coords_df = pd.merge(city_games_df,
                               worldcities_df,
                               left_on=['City'], 
                               right_on=['city_ascii'], 
                               how='left')

city_game_coords_df.head(10)

Get rows where coordinates are not set.

In [ ]:
city_game_coords_df[city_game_coords_df['lat'].isna()]

Correct the followings manually, one by one.
- Antwerpen → Antwerp
- Athina → Athens 
- Chamonix → Chamonix-Mont-Blanc
- Garmisch-Partenkirchen → Garmisch-Partenkirchen (47.492, 11.0931)
- Moskva → Moscow
- Sankt Moritz → St. Moritz (46.578, 9.8353)
- Squaw Valley → Palisades Tahoe (39.208, -120.2132)
- Torino → Turin

In [ ]:
city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Antwerpen"),
    ['city_ascii', 'lat', 'lng']] = worldcities_df.query('city_ascii == "Antwerp"').values

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Athina"),
    ['city_ascii', 'lat', 'lng']] = worldcities_df.query('city_ascii == "Athens"').values

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Chamonix"),
    ['city_ascii', 'lat', 'lng']] = worldcities_df.query('city_ascii == "Chamonix-Mont-Blanc"').values

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Garmisch-Partenkirchen"),
    ['city_ascii', 'lat', 'lng']] = ['Garmisch-Partenkirchen', 47.492, 11.0931]

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Moskva"),
    ['city_ascii', 'lat', 'lng']] = worldcities_df.query('city_ascii == "Moscow"').values

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Sankt Moritz"),
    ['city_ascii', 'lat', 'lng']] = ['St. Moritz', 46.578, 9.8353]

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Squaw Valley"),
    ['city_ascii', 'lat', 'lng']] = ['Palisades Tahoe', 39.208, -120.2132]

city_game_coords_df.loc[
    (city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Torino"),
    ['city_ascii', 'lat', 'lng']] = worldcities_df.query('city_ascii == "Turin"').values

Get rows where coordinates are not set again.

In [ ]:
city_game_coords_df[city_game_coords_df['lat'].isna()]

Get the max value of `number_of_times`.

In [ ]:
city_game_coords_df['number_of_times'].max()

Prepare colors for the circles.

In [ ]:
number_of_times_max = city_game_coords_df['number_of_times'].max()

# 夏季用
color_reds = sns.color_palette("Reds_r", number_of_times_max).as_hex()
# 冬季用
color_blues = sns.color_palette("Blues_r", number_of_times_max).as_hex()

# 色の確認
sns.palplot(color_reds)
sns.palplot(color_blues)

Put circles on a map.

Do you find anything interesting?

In [ ]:
map = folium.Map(location=[0,0], zoom_start=2.4)

# 古い円をクリックできるよう、新しいものから順に重ねて描画
for index, row in city_game_coords_df.sort_values('Year', ascending=False).iterrows():
    location = (row['lat'], row['lng'])
    radius = row['number_of_athletes'] * 10
    color = color_reds[row['number_of_times'] - 1] if row['season'] == 'Summer' \
                                                   else color_blues[row['number_of_times'] - 1]

    folium.Circle(location=location,
                  radius=radius,
                  color=color,
                  fill_color=color,
                  weight=1.5,
                  popup=f"{row['City']}\n{row['Year']}\n{row['season']}").add_to(map)
map

### 3. Digging a bit deeper
We won't go into details, it's ok if you just know the steps needed for the analysis. 

Pick up one particular country and see changes in how many and what kind of medals they've got.

In [ ]:
# 日本選手に限定
athlete_jpn_df = athlete_events_df.query('NOC == "JPN"')

# 年、メダルでグループ分け
medal_jpn_df = athlete_jpn_df.groupby(['Year', 'Medal']).agg({'ID':'count'}).reset_index()

medal_jpn_df.head()

Draw a line plot.

In [ ]:
# 銅の色はbrownで代用
sns.lineplot(data=medal_jpn_df,
             x='Year',
             y='ID',
             hue='Medal',
             hue_order=["Gold", "Silver", "Bronze"],
             palette=["gold", "silver", "brown"])

### 4. Exercise for visualizing data on a map
Search the Web and find the passenger traffic data of JR EAST or Tokyo Metro, then visualize it on a map.

How to get data on a Web page:  
Call `pd.read_html()` with a URL as an argument and you'll get DataFrame list, with each corresponding to each table on the page.  
If you get an error when reading the page, pass the HTML code as an argument instead as follows.

If you get a DataFrame with multi-level indexes which you don't want, call `pandas.MultiIndex.droplevel()` to remove unnecessary levels.

__pandas.MultiIndex.droplevel__  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.droplevel.html

In [ ]:
# HTMLソースを取得
response = requests.get('https://en.wikipedia.org/wiki/List_of_highest-grossing_films')
# 文字化けする場合は設定
response.encoding = response.apparent_encoding
# データフレームに変換
dfs = pd.read_html(response.text)
# 任意のデータフレームを選択
dfs[1]